In [0]:
%run ./config

In [0]:
%run ./utils

In [0]:
from pyspark.sql.functions import max, col,current_timestamp, to_date

In [0]:
spark.sql(f"CREATE DATABASE IF NOT EXISTS {catalog_database}")

In [0]:
vehicle_instance_path = f'{path_prefix}/vehicle_instance'
sensor_data_path = f'{path_prefix}/sensor_data_path'

date_stamp = spark.read.format("csv")\
                .load(vehicle_instance_path)\
                .select("loaded_dt").agg(max("loaded_dt")).collect()[0][0]

In [0]:
vehicle_instance_df = spark.read.format("csv").option("header", "true").load(vehicle_instance_path).filter(col("loaded_dt")==date_stamp)\
    .withColumn("__insert_time", current_timestamp())

sensor_data_df = spark.read.format("csv").option("header", "true").load(sensor_data_path).filter(col("loaded_dt")==date_stamp)\
    .withColumn("__insert_time", current_timestamp())

In [0]:
vehicle_instance_col_mapping = {
"vehicle_id" : "int",
"model_id" : "int",
"variant_id":"int",
"owner_id": "int"
}

sensor_data_col_mapping = {
"recorded_date" : "string",
"vehicle_id" : "int",
"sensor_a": "int",
"sensor_b": "int",
"sensor_c": "int",
"sensor_d": "int",
"sensor_e": "int",
"sensor_f": "int",
"sensor_g": "int",
"sensor_h": "int",
"loaded_dt": "long"
}


veh_casted_df = cast_cols(vehicle_instance_df,vehicle_instance_col_mapping)
sens_casted_df = cast_cols(sensor_data_df,sensor_data_col_mapping)
sens_date_casted_df = sens_casted_df.withColumn("recorded_date", to_date("recorded_date"))

sens_date_casted_df.write.mode("overwrite").saveAsTable(f"{catalog_database}.bronze_sensor_data")
veh_casted_df.write.mode("overwrite").saveAsTable(f"{catalog_database}.bronze_vehicle_data")

In [0]:
%sql
select * from kaninipro_catalog.dev.bronze_sensor_data 
where vehicle_id = 1
order by recorded_date

In [0]:
%sql
select count(*) from kaninipro_catalog.dev.bronze_sensor_data 
where vehicle_id = 1

In [0]:
%sql
select * from kaninipro_catalog.dev.bronze_vehicle_data